In [2]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.2 MB/s 


In [3]:
import numpy as np # linear algebra
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import catboost as cbt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import pipeline

In [4]:
df = pd.read_csv('data.csv')

df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [5]:
# mapping to the sentiment column 

dicto = {'positive': 1, 'neutral': 0 , 'negative': -1}

df.Sentiment = df.Sentiment.map(dicto)

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
ctv = CountVectorizer(tokenizer= word_tokenize, token_pattern=None)

ctv.fit_transform(df['Sentence'])

<5842x13215 sparse matrix of type '<class 'numpy.int64'>'
	with 111586 stored elements in Compressed Sparse Row format>

#Model-1 Logistic + BOW

In [9]:
from sklearn import linear_model
from sklearn import model_selection

df['kfold'] = -1

df = df.sample(frac=1).reset_index(drop= True)

y = df.Sentiment.values

# Intitiate kfold class from model_selection module

kf = model_selection.StratifiedKFold(n_splits = 5)

for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
# we go over the folds
for fold_ in range(5):
    train_df = df[df.kfold != fold_].reset_index(drop=True)
    test_df = df[df.kfold == fold_].reset_index(drop=True)
    count_vec.fit(train_df.Sentence)
    
    xtrain = count_vec.transform(train_df.Sentence)
    xtest = count_vec.transform(test_df.Sentence)
    
    model = linear_model.LogisticRegression(solver = 'liblinear')
    
    model.fit(xtrain,train_df.Sentiment)
    
    preds = model.predict(xtest)
    
    accuracy_precision = precision_score(test_df.Sentiment, preds, average='micro')
    accuracy_recall = recall_score(test_df.Sentiment, preds, average = 'micro')
    
    print('precision score:', accuracy_precision)
    print('recall score:', accuracy_recall)
    print("")
    
print(classification_report(test_df.Sentiment, preds, labels=[1, -1, 0]))

precision score: 0.6740804106073567
recall score: 0.6740804106073567

precision score: 0.6672369546621043
recall score: 0.6672369546621043

precision score: 0.6986301369863014
recall score: 0.6986301369863014

precision score: 0.696917808219178
recall score: 0.696917808219178

precision score: 0.690068493150685
recall score: 0.690068493150685

              precision    recall  f1-score   support

           1       0.82      0.75      0.78       370
          -1       0.25      0.21      0.23       172
           0       0.72      0.79      0.75       626

    accuracy                           0.69      1168
   macro avg       0.60      0.58      0.59      1168
weighted avg       0.68      0.69      0.68      1168



#Model 2 NaiveBayes + BOW

In [10]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))  

precision score: 0.6911890504704876

precision score: 0.7134302822925578

precision score: 0.702054794520548

precision score: 0.6866438356164384

precision score: 0.6857876712328768

              precision    recall  f1-score   support

           1       0.71      0.62      0.66       370
          -1       0.41      0.29      0.34       172
           0       0.72      0.83      0.77       626

    accuracy                           0.69      1168
   macro avg       0.61      0.58      0.59      1168
weighted avg       0.67      0.69      0.67      1168



#Model-3 Naive Bayes with TFIDF

In [11]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for f_,(t_,v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f_
    
for fold in range(5):
    df_train = df[df.kfold != fold].reset_index(drop = True)
    df_test = df[df.kfold == fold].reset_index(drop = True)
    
    tf_vec = TfidfVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    xtrain = tf_vec.fit_transform(df_train.Sentence)
    xtest = tf_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(xtrain, df_train.Sentiment)
    
    y_pred = model.predict(xtest)
    
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')

    print('precision score:', accuracy_precision)

    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    

precision score: 0.6432848588537211

precision score: 0.6415739948674081

precision score: 0.6558219178082192

precision score: 0.6549657534246576

precision score: 0.6429794520547946

              precision    recall  f1-score   support

           1       0.71      0.37      0.49       370
          -1       1.00      0.02      0.03       172
           0       0.63      0.98      0.76       626

    accuracy                           0.64      1168
   macro avg       0.78      0.45      0.43      1168
weighted avg       0.71      0.64      0.57      1168



#Model-4 Naive Bayes with Ngrams

In [12]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None, ngram_range = (1,3))
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))   

precision score: 0.6586826347305389

precision score: 0.6518391787852865

precision score: 0.6712328767123288

precision score: 0.6601027397260274

precision score: 0.6446917808219178

              precision    recall  f1-score   support

           1       0.76      0.58      0.66       370
          -1       0.10      0.05      0.07       172
           0       0.67      0.84      0.74       626

    accuracy                           0.64      1168
   macro avg       0.51      0.49      0.49      1168
weighted avg       0.61      0.64      0.62      1168



#Text processing

In [13]:
def process_text(text):
    
    text = word_tokenize(text)
    text = [re.sub('[^A-Za-z]+', '', word) for word in text]
    text = [word.lower() for word in text if word.isalpha()]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
text = 'He is not a good person'
token_text = word_tokenize(text)
[ word for word in token_text if word not in stopwords.words('english')]

['He', 'good', 'person']

#Model 5 cleaned text Naive Bayes BOW

In [17]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB()
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))    

precision score: 0.6928999144568007

precision score: 0.7159965782720273

precision score: 0.7071917808219178

precision score: 0.6592465753424658

precision score: 0.7063356164383562

              precision    recall  f1-score   support

           1       0.77      0.63      0.69       370
          -1       0.45      0.31      0.37       172
           0       0.72      0.86      0.78       626

    accuracy                           0.71      1168
   macro avg       0.65      0.60      0.61      1168
weighted avg       0.70      0.71      0.69      1168



#Model 6 Fastext vector with Naive Bayes Baseline

In [22]:
import io

def sentence_to_vec(s, embedding_dict, tokenizer):
    
    words = tokenizer(s)
    
    embedding_list = []
    for w in words:
        
        if w in embedding_dict:
            embedding_list.append(embedding_dict[w])
            
    # if we dont have any vectors, then return zeros
    if len(embedding_list) == 0:
        return np.zeros(300)
    
    #convert list of embeddings into an array
    embedding_list = np.array(embedding_list)
    print
    #calculate sum over axis = 0 (This will convert the word vectorrs to sentence vectors)
    v = embedding_list.sum(axis=0)
    
    #return normalized vector
    return v

df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df.drop('kfold', axis=1, inplace = True)

kf = model_selection.StratifiedKFold(n_splits = 5)

#load embeddings into memory
print("Loading embeddings")
embeddings = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/AI/NLP/SentimentAnalysis/crawl-300d-2M.vec')
    
#create sentence embeddings
print("creating sentence vectors")
vectors = []
for sentence in df.Sentence.values:
    vectors.append(         
       sentence_to_vec(s = sentence, embedding_dict= embeddings, tokenizer = word_tokenize)
    )
vectors = np.array(vectors)
    
for fold_,(train_, valid_) in enumerate(kf.split(X=df, y=y)):
        print("fold: ", fold_)
        
        xtrain = vectors[train_,:]
        ytrain = y[train_]
        
        xtest = vectors[valid_,:]
        ytest = y[valid_]
        
        scaler = MinMaxScaler()
        
        xscaled_train = scaler.fit_transform(xtrain)
        xscaled_test = scaler.transform(xtest)
        
        model = MultinomialNB()
    
        model.fit(xscaled_train,ytrain)
    
        y_pred = model.predict(xscaled_test)
    
        pres_score = precision_score(ytest, y_pred, average='micro')
    
        rec_score = recall_score(ytest, y_pred, average='micro')
    
        print('precision and recall scores:', pres_score, rec_score)
        print("")
    
print(classification_report(ytest, y_pred, labels=[1,-1,0]))

Loading embeddings
creating sentence vectors
fold:  0
precision and recall scores: 0.5414884516680923 0.5414884516680923

fold:  1
precision and recall scores: 0.542343883661249 0.542343883661249

fold:  2
precision and recall scores: 0.5402397260273972 0.5402397260273972

fold:  3
precision and recall scores: 0.5419520547945206 0.5419520547945206

fold:  4
precision and recall scores: 0.5428082191780822 0.5428082191780822

              precision    recall  f1-score   support

           1       0.48      0.03      0.05       370
          -1       0.00      0.00      0.00       172
           0       0.54      1.00      0.70       626

    accuracy                           0.54      1168
   macro avg       0.34      0.34      0.25      1168
weighted avg       0.44      0.54      0.39      1168



#Model # Hyperparameter tuning best models

In [18]:
from sklearn.model_selection import GridSearchCV

count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)

X_train = count_vec.fit_transform(df.Sentence)

parameters = {'alpha': [0.001,0.01,0.1,0.2,0.3,0.5,0.7,1,1.5,1.6,1.8,10,100]}

grid_search = MultinomialNB()

grid_search = GridSearchCV(grid_search, parameters, cv=5, scoring = 'f1_micro', n_jobs = -1)

grid_result = grid_search.fit(X_train, df.Sentiment)

print('Best params: ', grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

Best params:  {'alpha': 1.6}
Best score:  0.7007876126416444


In [19]:

df = df.sample(frac=1).reset_index(drop=True)



count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern = None)

X_train = count_vec.fit_transform(df.Sentence)

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv=5, scoring = 'f1_micro', n_jobs = -1)
 
# fitting the model for grid search
grid.fit(X_train, df.Sentiment)

print('Best params: ', grid.best_params_)
print('Best score: ', grid.best_score_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best params:  {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best score:  0.7031844334813739


#Model 8 Best basline model+hyperparameters

In [20]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = MultinomialNB(alpha = 1.6)
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))   

precision score: 0.7074422583404619

precision score: 0.6903336184773311

precision score: 0.6618150684931506

precision score: 0.7063356164383562

precision score: 0.6926369863013698

              precision    recall  f1-score   support

           1       0.72      0.62      0.67       370
          -1       0.40      0.13      0.20       172
           0       0.70      0.89      0.79       626

    accuracy                           0.69      1168
   macro avg       0.61      0.55      0.55      1168
weighted avg       0.66      0.69      0.66      1168



In [21]:
df = df.sample(frac=1).reset_index(drop=True)
y = df.Sentiment.values

df['kfold'] = -1

kf = model_selection.StratifiedKFold(n_splits = 5)

for fold_ ,(t_,v_) in enumerate(kf.split(X= df, y=y)):
    df.loc[v_, 'kfold'] = fold_
    
for f in range(5):
    df_train = df[df.kfold != f].reset_index(drop=True)
    df_test = df[df.kfold == f].reset_index(drop=True)
    
    count_vec = CountVectorizer(tokenizer = word_tokenize, token_pattern=None)
    
    x_train = count_vec.fit_transform(df_train.Sentence)
    x_test = count_vec.transform(df_test.Sentence)
    
    model = SVC(C= 100, gamma =  0.001, kernel = 'rbf')
    
    model.fit(x_train, df_train.Sentiment)
    
    y_pred = model.predict(x_test)
    
    accuracy_precision = precision_score(df_test.Sentiment, y_pred, average = 'micro')
    
    #accuracy_recall = recall_score(df_test.Sentiment, y_pred)
    
    print('precision score:', accuracy_precision)
    #print('recall score:', accuracy_recall)
    print("")
    

print(classification_report(df_test.Sentiment, y_pred, labels=[1,-1,0]))   

precision score: 0.6663815226689478

precision score: 0.6937553464499572

precision score: 0.6806506849315068

precision score: 0.6772260273972602

precision score: 0.7054794520547946

              precision    recall  f1-score   support

           1       0.79      0.75      0.77       370
          -1       0.31      0.23      0.26       172
           0       0.73      0.81      0.77       626

    accuracy                           0.71      1168
   macro avg       0.61      0.60      0.60      1168
weighted avg       0.69      0.71      0.70      1168

